##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# 如何使用 TF-Hub 构建简单的文本分类器


> 注：本教程使用**已弃用的** TensorFlow 1 功能。有关完成此任务的新方式，请参阅 [TensorFlow 2 版本](https://tensorflow.google.cn/hub/tutorials/tf2_text_classification)。


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/hub/tutorials/text_classification_with_tf_hub.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/hub/tutorials/text_classification_with_tf_hub.ipynb">     <img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">     在 GitHub 上查看源代码</a></td>
  <td><a href="https://tfhub.dev/google/nnlm-en-dim128/1"><img src="https://tensorflow.google.cn/images/hub_logo_32px.png">查看 TF Hub 模型</a></td>
</table>

TF-Hub 是一个共享打包在可重用资源（尤其是预训练的**模块**）中的机器学习专业知识的平台。本教程分为两个主要部分。

**入门**：使用 TF-Hub 训练文本分类器

我们将使用 TF-Hub 文本嵌入向量模块训练具有合理基线准确率的简单情感分类器。然后，我们将分析预测结果以确保模型合理，并提出改进措施以提高准确率。

**高级：**迁移学习分析

在本部分中，我们将使用各种 TF-Hub 模块来比较它们对 Estimator 准确率的影响，并展示迁移学习的优势和缺陷。


## 可选前提条件

- 对 Tensorflow [预制 Estimator 框架](https://tensorflow.google.cn/get_started/premade_estimators)有基本了解。
- 熟悉 [Pandas](https://pandas.pydata.org/) 库。


## 设置

In [ ]:
# Install TF-Hub.
!pip install seaborn

有关安装 Tensorflow 的更多详细信息，请访问 [https://tensorflow.google.cn/install/](https://tensorflow.google.cn/install/)。

In [ ]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

# 开始

## 数据

我们将尝试解决 [Large Movie Review Dataset v1.0](http://ai.stanford.edu/~amaas/data/sentiment/) 任务[（Mass 等人，2011 年）](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf)。数据集由 IMDB 电影评论组成，这些评论使用从 1 到 10 的正数标记。任务是将评论标记为**负面**或**正面**。

In [ ]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()

## 模型

### 输入函数

[Estimator 框架](https://tensorflow.google.cn/get_started/premade_estimators#overview_of_programming_with_estimators)提供了封装 Pandas 数据帧的[输入函数](https://tensorflow.google.cn/api_docs/python/tf/compat/v1/estimator/inputs/pandas_input_fn)。

In [ ]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    test_df, test_df["polarity"], shuffle=False)

### 特征列

TF-Hub 提供了一个[特征列](https://tensorflow.google.cn/hub/api_docs/python/hub/text_embedding_column.md)，此列在给定的文本特征上应用模块，并进一步传递模块的输出。在本教程中，我们将使用 [nnlm-en-dim128 模块](https://tfhub.dev/google/nnlm-en-dim128/1)。对于本教程而言，最重要的事实如下：

- 模块将**字符串的一维张量中的一批句子**作为输入。
- 模块负责**句子的预处理**（例如，移除标点符号和在空格处拆分）。
- 模块可以使用任何输入（例如，**nnlm-en-dim128** 将词汇中不存在的单词散列到约 20000 个桶中）。

In [ ]:
embedded_text_feature_column = hub.text_embedding_column(
    key="sentence", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

### Estimator

要实现分类，我们可以使用 [DNN 分类器](https://tensorflow.google.cn/api_docs/python/tf/estimator/DNNClassifier)（请注意本教程结尾处有关标签函数的不同建模的补充说明）。

In [ ]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.keras.optimizers.Adagrad(lr=0.003))

### 训练

以合理的步骤数训练 Estimator。

In [ ]:
# Training for 5,000 steps means 640,000 training examples with the default
# batch size. This is roughly equivalent to 25 epochs since the training dataset
# contains 25,000 examples.
estimator.train(input_fn=train_input_fn, steps=5000);

# 预测

为训练集和测试集运行预测。

In [ ]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))

## 混淆矩阵

我们可以目视检查混淆矩阵，以了解错误分类的分布。

In [ ]:
def get_predictions(estimator, input_fn):
  return [x["class_ids"][0] for x in estimator.predict(input_fn=input_fn)]

LABELS = [
    "negative", "positive"
]

# Create a confusion matrix on training data.
cm = tf.math.confusion_matrix(train_df["polarity"], 
                              get_predictions(estimator, predict_train_input_fn))

# Normalize the confusion matrix so that each row sums to 1.
cm = tf.cast(cm, dtype=tf.float32)
cm = cm / tf.math.reduce_sum(cm, axis=1)[:, np.newaxis]

sns.heatmap(cm, annot=True, xticklabels=LABELS, yticklabels=LABELS);
plt.xlabel("Predicted");
plt.ylabel("True");

# 进一步改进

1. **情感回归**：我们使用分类器将每个样本分配给一个极性类。但实际上，我们还有另一个分类特征 - 情感。在这里，类实际上表示一个比例，并且基础值（正/负）可以很好地映射到连续范围内。我们可以通过计算回归（[DNN 回归器](https://tensorflow.google.cn/api_docs/python/tf/contrib/learn/DNNRegressor)）而不是分类（[DNN 分类器](https://tensorflow.google.cn/api_docs/python/tf/contrib/learn/DNNClassifier)）来利用此属性。
2. **较大的模块**：对于本教程而言，我们使用了较小的模块来限制内存使用。有些模块具有更大的词汇和更大的嵌入向量空间，可以提供更多的准确率点。
3. **参数调节**：我们可以通过调节元参数（例如学习率或步骤数）来提高准确率，尤其是在使用不同模块的情况下。如果我们想获得任何合理的结果，那么验证集非常重要，因为这样可以轻松建立一个模型来学习预测训练数据，而无需很好地泛化到测试集。
4. **更复杂的模型**：我们使用了一个通过嵌入每个单词并随后将其与平均值相结合来计算句子嵌入向量的模块。此外，也可以使用序贯模块（例如 [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder/2) 模块）来更好地捕获句子的性质。或者，使用两个或多个 TF-Hub 模块的集合。
5. **正则化**：为了避免过拟合，我们可以尝试使用执行某种正则化的优化器，例如[近端 Adagrad 优化器](https://tensorflow.google.cn/api_docs/python/tf/compat/v1/train/ProximalAdagradOptimizer)。


# 高级：迁移学习分析

迁移学习可以**节省训练资源**，即使**基于小数据集训练**也可以实现良好的模型泛化。在这一部分中，我们将通过使用两个不同的 TF-Hub 模块进行训练来演示这一点：

- **[nnlm-en-dim128](https://tfhub.dev/google/nnlm-en-dim128/1)** - 预训练的文本嵌入向量模块；
- **[random-nnlm-en-dim128](https://tfhub.dev/google/random-nnlm-en-dim128/1)** - 文本嵌入向量模块，其词汇和网络与 **nnlm-en-dim128** 相同，但权重只是随机初始化的，从未基于真实数据进行训练。

在以下两种模式下训练：

- **仅训练分类器**（即冻结模块），以及
- **将分类器与模块一起训练**。

我们运行一些训练和评估来查看使用各种模块如何影响准确率。

In [ ]:
def train_and_evaluate_with_module(hub_module, train_module=False):
  embedded_text_feature_column = hub.text_embedding_column(
      key="sentence", module_spec=hub_module, trainable=train_module)

  estimator = tf.estimator.DNNClassifier(
      hidden_units=[500, 100],
      feature_columns=[embedded_text_feature_column],
      n_classes=2,
      optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.003))

  estimator.train(input_fn=train_input_fn, steps=1000)

  train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
  test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

  training_set_accuracy = train_eval_result["accuracy"]
  test_set_accuracy = test_eval_result["accuracy"]

  return {
      "Training accuracy": training_set_accuracy,
      "Test accuracy": test_set_accuracy
  }


results = {}
results["nnlm-en-dim128"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/nnlm-en-dim128/1")
results["nnlm-en-dim128-with-module-training"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/nnlm-en-dim128/1", True)
results["random-nnlm-en-dim128"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/random-nnlm-en-dim128/1")
results["random-nnlm-en-dim128-with-module-training"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/random-nnlm-en-dim128/1", True)

我们来看看结果。

In [ ]:
pd.DataFrame.from_dict(results, orient="index")

我们已经看到了一些模式，但首先我们应当建立测试集的基线准确率 - 通过仅输出最具代表性的类的标签可以实现的下限：

In [ ]:
estimator.evaluate(input_fn=predict_test_input_fn)["accuracy_baseline"]

分配最具代表性的类将使我们的准确率达到 **50%**。这里有几件事需要注意：

1. 也许会令人惊讶，但此时**仍然可以在固定的随机嵌入向量上学习模型**。原因是，即使字典中的每个单词都映射到随机向量，Estimator 也可以仅仅使用其全连接层来分隔空间。
2. 与仅训练分类器相比，允许使用**随机嵌入向量**训练模块有助于提高训练和测试的准确率。
3. 使用**预训练的嵌入向量**训练模块也有助于提高这两个准确率。不过，此时需要注意在训练集上的过拟合。即使进行正则化，训练一个预训练的模块也可能存在风险，因为嵌入向量权重不再表示基于各种数据训练的语言模型，相反，它们会收敛到新数据集的理想表示。